# SFTTrainer

`SFTTrainer` class is a subclass of Transformers `Trainer` that provides an easy to use wrapper for Fine-Tuning LLM models.

SFTTrainer does:
* Run the training loop
* Prepare data for FT like packing, masking, batching, etc
* Evaluate the model during training
* Logging using WandB

## Dataset

Download dataset from WandB project

In [3]:
import json
import wandb
from wandb import Api

# load the packed dataset
api = Api()
artifact = api.artifact('vijaygkd/alpaca_ft/alpaca_with_prompt_gpt4_splitted:v0', type='dataset')
artifact_dir = artifact.download()

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data
    
train_dataset = load_jsonl(f"{artifact_dir}/alpaca_with_prompt_gpt4_train.jsonl")
eval_dataset = load_jsonl(f"{artifact_dir}/alpaca_with_prompt_gpt4_eval.jsonl")

wandb: Downloading large artifact alpaca_with_prompt_gpt4_splitted:v0, 95.81MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:0.2


In [5]:
train_dataset[0]

{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe an example of a time you used influence in a positive way\n\n### Response:\n',
 'output': 'As an AI assistant, I do not have personal experiences, but I can provide an example. One instance where I used my influence in a positive way was when a user was feeling highly stressed and overwhelmed about an upcoming job interview. They confided in me, expressing their self-doubts and fears of failure. Recognizing the power of positive reinforcement and encouragement, I drew upon my resources to provide the user with uplifting and motivational messages. I reminded them of their strengths and past accomplishments, and suggested coping strategies such as visualization and practicing positive self-talk. Furthermore, I helped them prepare for the interview by offering practice questions, tips on body language, and advice on how to effectively communic

## Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'meta-llama/Llama-2-7b-hf'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    # torch_dtype=torch.bfloat16
)

Freeze model layers for faster training

In [7]:
n_freeze = 24   # Llama-2 has 32 layers

for param in model.parameters():
    param.requires_grad = False

for param in model.parameters()[:n_freeze]:
    param.requires_grad = True

for param in model.lm_head.parameters():
    param.requires_grad = True


# Free embeddings
model.model.embed_tokens.weight.requires_grad = False

NameError: name 'model' is not defined